# Simple multilayer neural network

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

In [3]:
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [4]:
iris = datasets.load_iris()
X = normalize(iris.data)
y = to_one_hot(iris.target)

In [5]:
order = np.random.permutation(len(X))
portion = 20
test_x = X[order[:portion]]
test_y = y[order[:portion], :]
train_x = X[order[portion:]]
train_y = y[order[portion:], :]

In [8]:
class Classifier:
    
    def __init__(self, layers_sizes, lr, verbose):
        self.l_sizes = layers_sizes
        self.lr = lr
        self.verbose = verbose

    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def sigmoid_derivative(self, x):
        return self.__sigmoid(x) * (1 - self.__sigmoid(x))

    def fit(self, X, y):
        self.w_1 = 2*np.random.random((X.shape[1], self.l_sizes[0]))        
        self.w_2 = 2*np.random.random((self.l_sizes[0], self.l_sizes[1]))
        
        errors = []
        
        for i in range(10000):
            layer1 =  self.__sigmoid(X @ self.w_1)
            layer2 =  self.__sigmoid(layer1 @ self.w_2)

            layer2_error = y - layer2
            layer2_delta = layer2_error * self.sigmoid_derivative(layer2)

            layer1_error = layer2_delta @ self.w_2.T
            layer1_delta = layer1_error * self.sigmoid_derivative(layer1)
            
            self.w_2 += layer1.T.dot(layer2_delta) * self.lr
            self.w_1 += X.T.dot(layer1_delta) * self.lr
            
            error = np.mean(np.abs(layer2_error))
            errors.append(error)
            accuracy = (1 - error) * 100
        
            if(self.verbose == True and i % 1000 == 0):
                print("Iteration: %d - Accuracy: %.4f" %(i, accuracy))
                
    def predict(self, X):
        layer1 =  self.__sigmoid(X @ self.w_1)
        return self.__sigmoid(layer1 @ self.w_2)

In [9]:
model = Classifier((5,3), 0.1, verbose = True)
model.fit(train_x, train_y)

Iteration: 0 - Accuracy: 33.7203
Iteration: 1000 - Accuracy: 93.6033
Iteration: 2000 - Accuracy: 95.6988
Iteration: 3000 - Accuracy: 96.3381
Iteration: 4000 - Accuracy: 97.1540
Iteration: 5000 - Accuracy: 97.4209
Iteration: 6000 - Accuracy: 97.5865
Iteration: 7000 - Accuracy: 97.6959
Iteration: 8000 - Accuracy: 97.7746
Iteration: 9000 - Accuracy: 97.8358


In [11]:
layer2_error = test_y - model.predict(test_x)

error = np.mean(np.abs(layer2_error))
accuracy = (1 - error) * 100

print("Validation Accuracy " + str(round(accuracy,2)) + "%")

Validation Accuracy 96.1%
